## LinkedIn Profile Visit and Connection Request

In [147]:
import os, time
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [148]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')
driver.title

'LinkedIn Login, Sign in | LinkedIn'

In [149]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.environ['EMAIL'])

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ['PASSWORD'])

password.submit()

In [150]:

url = "https://www.linkedin.com/company/siemens/people/?facetGeoRegion=101282230"
driver.get(url)

In [153]:
def visit_and_connect(driver, profile_url, timeout=5):
    """
    Visit a LinkedIn profile and attempt to connect or follow.
    
    Args:
        driver: Selenium WebDriver instance
        profile_url: LinkedIn profile URL
        timeout: Wait timeout in seconds (default: 5)
    
    Returns:
        str: Action taken ('connect', 'follow', 'already_pending', 
             'already_following', 'already_connected', 'none_found')
    """
    driver.get(profile_url)
    wait = WebDriverWait(driver, timeout)
    time.sleep(2)
    action_taken = None

    # Check if already connected or following
    try:
        # Check for "Pending" button (connection request already sent)
        pending_btn = driver.find_element(By.XPATH, "//main//button[.//span[normalize-space()='Pending']]")
        print("⏳ Connection request already sent (Pending)")
        action_taken = 'already_pending'
        # return action_taken
    except NoSuchElementException:
        pass

    # Check for "Message" button without Connect/Follow (already connected)
    if action_taken is None:
        try:
            message_btn = driver.find_element(By.XPATH, "//main//button[.//span[normalize-space()='Message']]")
            # Check if Connect or Follow buttons exist
            connect_exists = len(driver.find_elements(By.XPATH, "//main//button[.//span[normalize-space()='Connect']]")) > 0
            follow_exists = len(driver.find_elements(By.XPATH, "//main//button[.//span[normalize-space()='Follow']]")) > 0
            
            if not connect_exists and not follow_exists:
                print("✓ Already connected with this person")
                action_taken = 'already_connected'
                # return action_taken
        except NoSuchElementException:
            pass

    # Only proceed if no action has been taken yet
    if action_taken is None:
        # Try Connect button
        try:
            connect_btn = wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//main//button[.//span[normalize-space()='Connect']]")
                )
            )
            # print(f"📤 Connect button found")
            # connect_btn.click()
            # print("   Connect button clicked")
            
            # Handle the "Add a note" modal
            # try:
            #     send_without_note_btn = wait.until(
            #         EC.element_to_be_clickable(
            #             (By.XPATH, "//button[.//span[normalize-space()='Send without a note']]")
            #         )
            #     )
            #     send_without_note_btn.click()
            #     print("✓ Sent connection request without a note")
            #     time.sleep(2)
            #     action_taken = 'connect'
                
            # except TimeoutException:
            #     print("⚠️ 'Send without a note' button not found")
            #     # Close modal if it exists
            #     try:
            #         close_btn = driver.find_element(By.XPATH, "//button[@aria-label='Dismiss']")
            #         close_btn.click()
            #     except:
            #         pass
            
        except TimeoutException:
            print("   Connect button not found in main profile")
            
        # Try Follow button
        try:
            follow_btn = wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//main//button[.//span[normalize-space()='Follow']]")
                )
            )
            follow_btn.click()
            print("✓ Follow button clicked successfully")
            action_taken = 'follow'
            
        except TimeoutException:
            print("❌ Follow button not found either")
            action_taken = 'none_found'

    # # Summary
    # print(f"ACTION SUMMARY: {action_taken}")

In [1]:
# Store the main window handle
main_window = driver.current_window_handle
seen_urls = set()

while True:
    # collect profiles currently loaded
    profile_links = driver.find_elements(
        By.XPATH,
        "//a[contains(@href, 'linkedin.com/in/')]"
    )
    
    # Extract all URLs first and remove duplicates
    profile_urls = []
    for link in profile_links:
        href = link.get_attribute("href")
        if href:
            # Clean URL: remove query parameters
            clean_url = href.split('?')[0].rstrip('/')
            if clean_url not in seen_urls:
                profile_urls.append(clean_url)
    
    print(f"Found {len(profile_urls)} new profiles to visit")
    
    # Now visit each URL in a new tab
    for url in profile_urls:
        if url in seen_urls:
            print("@@x@@ Already seen continue ...")
            continue
        print(f"\n{'='*50}")
        print("Visiting Profile:", url)
        print(f"{'='*50}")

        # Open profile in new tab
        driver.execute_script("window.open('');")
        
        # Switch to new tab
        driver.switch_to.window(driver.window_handles[-1])
        
        # Now navigate to the profile
        visit_and_connect(driver, url)
        time.sleep(2)
        
        # Close tab and switch back to main window
        driver.close()
        driver.switch_to.window(main_window)
        time.sleep(1)
        # Mark as seen
        seen_urls.add(url)
        
        # input("Press Enter to continue...")
    
    # try clicking "Show more results"
    try:
        show_more = driver.find_element(
            By.XPATH,
            "//button[.//span[text()='Show more results']]"
        )
        driver.execute_script("arguments[0].click();", show_more)
        time.sleep(2)
    except NoSuchElementException:
        break

NameError: name 'driver' is not defined